In [1]:
# !pip install lightfm

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Для работы с матрицами
from scipy.sparse import csr_matrix, coo_matrix

# Матричная факторизация
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from lightfm import LightFM

import warnings
warnings.simplefilter("ignore")

In [3]:
from lightfm.evaluation import precision_at_k, recall_at_k

from src.metrics import precision_at_k as custom_precision
from src.utils import prefilter_items

In [4]:
data = pd.read_csv('./data/retail_train.csv')

item_features = pd.read_csv('./data/product.csv')
user_features = pd.read_csv('./data/hh_demographic.csv')

# column processing
item_features.columns = [col.lower() for col in item_features.columns]
user_features.columns = [col.lower() for col in user_features.columns]

item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
user_features.rename(columns={'household_key': 'user_id'}, inplace=True)

# train test split
test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

data_train.head(2)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0


In [5]:
item_features.head(2)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,


In [6]:
user_features.head(2)

,age_desc,marital_status_code,income_desc,homeowner_desc,hh_comp_desc,household_size_desc,kid_category_desc,user_id
0,65+,A,35-49K,Homeowner,2 Adults No Kids,2,None/Unknown,1
1,45-54,A,50-74K,Homeowner,2 Adults No Kids,2,None/Unknown,7


In [7]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(2)

,user_id,actual
0,1,"[821867, 834484, 856942, 865456, 889248, 90795..."
1,3,"[835476, 851057, 872021, 878302, 879948, 90963..."


In [8]:
n_items_before = data_train['item_id'].nunique()

data_train_filtered = prefilter_items(data_train, take_n_popular=5000, item_features=item_features)

n_items_after = data_train_filtered['item_id'].nunique()
print('Decreased # items from {} to {}'.format(n_items_before, n_items_after))

Decreased # items from 86865 to 5001


In [9]:
user_item_matrix = pd.pivot_table(data_train_filtered, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', # Можно пробоват ьдругие варианты
                                  aggfunc='count', 
                                  fill_value=0
                                 )

user_item_matrix = user_item_matrix.astype(float) # необходимый тип матрицы для implicit

# переведем в формат sparse matrix
sparse_user_item = csr_matrix(user_item_matrix).tocsr()

In [10]:
data_test = data_test[data_test['item_id'].isin(data_train['item_id'].unique())]

test_user_item_matrix = pd.pivot_table(data_test, 
                                  index='user_id', columns='item_id', 
                                  values='quantity', 
                                  aggfunc='count', 
                                  fill_value=0
                                 )

test_user_item_matrix = test_user_item_matrix.astype(float) 

In [11]:
userids = user_item_matrix.index.values
itemids = user_item_matrix.columns.values

matrix_userids = np.arange(len(userids))
matrix_itemids = np.arange(len(itemids))

id_to_itemid = dict(zip(matrix_itemids, itemids))
id_to_userid = dict(zip(matrix_userids, userids))

itemid_to_id = dict(zip(itemids, matrix_itemids))
userid_to_id = dict(zip(userids, matrix_userids))

In [12]:
user_feat = pd.DataFrame(user_item_matrix.index)
user_feat = user_feat.merge(user_features, on='user_id', how='left')
user_feat.set_index('user_id', inplace=True)

In [13]:
item_feat = pd.DataFrame(user_item_matrix.columns)
item_feat = item_feat.merge(item_features, on='item_id', how='left')
item_feat.set_index('item_id', inplace=True)

In [14]:
user_feat_lightfm = pd.get_dummies(user_feat, columns=user_feat.columns.tolist())
item_feat_lightfm = pd.get_dummies(item_feat, columns=item_feat.columns.tolist())

In [15]:
model = LightFM(no_components=40,
                loss='bpr', # "logistic","bpr"
                learning_rate=0.01, 
                item_alpha=0.4,
                user_alpha=0.1, 
                random_state=42,
                k=5,
                n=15,
                max_sampled=100)

In [16]:
model.fit((sparse_user_item > 0) * 1, 
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=10, 
          num_threads=20,
          verbose=False) 

In [17]:
users_ids_row = data_train_filtered['user_id'].apply(lambda x: userid_to_id[x]).values.astype(int)
items_ids_row = data_train_filtered['item_id'].apply(lambda x: itemid_to_id[x]).values.astype(int)

In [18]:
predictions = model.predict(user_ids=users_ids_row,
                            item_ids=items_ids_row,
                            user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                            item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                            num_threads=10)

In [19]:
data_train_filtered['score'] = predictions

In [20]:
predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
            unique().reset_index()

In [21]:
df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')

In [22]:
precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
print(f"Precision: {precision}")

Precision: 0.14151067323480931


### Grid search

In [23]:
param = {
    'no_components': [10, 20, 40],
    'learning_rate': [0.01, 0.05, 0.1],
    'item_alpha': [0.0, 0.2, 0.4],
    'user_alpha': [0.0, 0.1, 0.2],
    'k': [2, 5, 10],
    'n': [5, 10, 15]
}

In [24]:
combined_params = np.array(np.meshgrid(param['no_components'], 
                                       param['learning_rate'], 
                                       param['item_alpha'],
                                       param['user_alpha'],
                                       param['k'],
                                       param['n'])).T.reshape(-1,6)

In [34]:
%%time
# прогоним нашу модель с этими параметрами и выберем лучший
result_list = []
counter = 0
for par in combined_params:
    model = LightFM(no_components=int(par[0]),
                    loss='bpr', # "logistic","bpr"
                    learning_rate=par[1], 
                    item_alpha=par[2],
                    user_alpha=par[3], 
                    random_state=42,
                    k=int(par[4]),
                    n=int(par[5]),
                    max_sampled=100)
    
    model.fit((sparse_user_item > 0) * 1, 
          sample_weight=coo_matrix(user_item_matrix),
          user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
          item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
          epochs=10, 
          num_threads=20,
          verbose=False)
    
    data_train_filtered['score'] = model.predict(user_ids=users_ids_row,
                                                 item_ids=items_ids_row,
                                                 user_features=csr_matrix(user_feat_lightfm.values).tocsr(),
                                                 item_features=csr_matrix(item_feat_lightfm.values).tocsr(),
                                                 num_threads=10)
    predict_result = data_train_filtered[['user_id','item_id','score']][data_train_filtered.item_id != 999999].drop_duplicates().sort_values(by=['user_id','score'], ascending=False).groupby('user_id')['item_id']. \
            unique().reset_index()
    df_result_for_metrics = result.merge(predict_result, on='user_id', how='inner')
    precision = df_result_for_metrics.apply(lambda row: custom_precision(row['item_id'], row['actual'],k=5), axis=1).mean()
    result_list.append((par, precision))
    counter += 1
    if counter % 50 == 0:
        print(f'{100 * counter / combined_params.shape[0]:.3f}%')

6.859%
13.717%
20.576%
27.435%
34.294%
41.152%
48.011%
54.870%
61.728%
68.587%
75.446%
82.305%
89.163%
96.022%
CPU times: user 11h 56min 48s, sys: 1h 49min 45s, total: 13h 46min 33s
Wall time: 1h 56min 19s


In [35]:
best_result = sorted(result_list, key=lambda x: x[1], reverse=True)[0]

In [36]:
print(f'no_components: {int(best_result[0][0])}\n'
      f'learning_rate: {best_result[0][1]}\n'
      f'item_alpha: {int(best_result[0][2])}\n'
      f'user_alpha: {int(best_result[0][3])}\n'
      f'k: {int(best_result[0][4])}\n'
      f'n: {int(best_result[0][5])}\n'
      f'result: {best_result[1]}')

no_components: 40
learning_rate: 0.01
item_alpha: 0
user_alpha: 0
k: 2
n: 5
result: 0.15993431855500606


In [1]:
# 0.15993431855500606 - 0.14151067323480931 = 0.018423645320196746